# HR Overtime Prediction and Forecasting

# Predicción y Pronóstico de Horas Extra de RRHH

Este notebook implementa el pronóstico de horas extra utilizando Prophet y visualiza datos históricos, predicciones e intervalos de confianza usando Plotly. Los componentes principales incluyen:
- Carga de datos históricos de horas extra desde SQL Server
- Entrenamiento de modelos SARIMA para cada departamento
- Generación de predicciones con intervalos de confianza 
- Visualización interactiva con Plotly
- Almacenamiento de modelos y predicciones en MS SQL SERVER

Objetivo: Predecir las horas extras acumuladas por departamento semana a semana para las próximas 4 semanas.

## 1. Importar Librerías and Setup


In [139]:
import pandas as pd
import numpy as np
import pymssql
import logging
import datetime
import warnings
import os

# Data visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Modelos y Tests
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from pmdarima import auto_arima

# Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

# Persistencia de modelo
import joblib

# Configure warnings and logging
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)

## 2. Conectar a la base de datos

Setup de SQL Server

In [140]:
# SQL Server Setup
SQL_SERVER = "172.28.192.1:50121"
SQL_DB = "HR_Analytics"
SQL_USER = "sa"
SQL_PASSWORD = "123456"

# Conectar a SQL Server
def get_db_connection():
    server_name = SQL_SERVER
    try:
        conn = pymssql.connect(
            server=server_name,
            database=SQL_DB,
            user=SQL_USER,
            password=SQL_PASSWORD
        )
        return conn
    except Exception as e:
        logging.error(f"Error de conexión a la base de datos: {e}")
        raise

## 3. Cargar y procesar datos históricos de horas extras

Consulta y preparación

In [141]:

def load_historical_data():
    conn = get_db_connection()
    query = "SELECT work_date, department, total_overtime FROM vw_historical_data ORDER BY work_date, department ASC"
    
    try:
        df = pd.read_sql(query, conn)
        print("Raw data record count:", len(df))
        print("Unique departments:", df['department'].unique())
        print("Null departments:", df['department'].isna().sum())
        print("Unique work_date values:", sorted(df['work_date'].unique()))
        
        # Convert work_date to datetime and strip time component
        df['work_date'] = pd.to_datetime(df['work_date']).dt.date
        df['work_date'] = pd.to_datetime(df['work_date'])
        
        print("Final record count:", len(df))
        print("Unique work_date and department combinations:", 
              df[['work_date', 'department']].drop_duplicates().shape[0])
        
        logging.info(f"Loaded data: {len(df)} records")
        return df
    
    finally:
        conn.close()

# Load and display data
historical_data = load_historical_data()
print("Muestra de datos históricos:")
display(historical_data.tail(12).sort_values(by=['work_date', 'department'], ascending=False))

INFO:root:Loaded data: 318 records


Raw data record count: 318
Unique departments: ['Finance' 'HR' 'Inventory' 'IT' 'Marketing' 'Sales']
Null departments: 0
Unique work_date values: [Timestamp('2024-05-19 00:00:00'), Timestamp('2024-05-26 00:00:00'), Timestamp('2024-06-02 00:00:00'), Timestamp('2024-06-09 00:00:00'), Timestamp('2024-06-16 00:00:00'), Timestamp('2024-06-23 00:00:00'), Timestamp('2024-06-30 00:00:00'), Timestamp('2024-07-07 00:00:00'), Timestamp('2024-07-14 00:00:00'), Timestamp('2024-07-21 00:00:00'), Timestamp('2024-07-28 00:00:00'), Timestamp('2024-08-04 00:00:00'), Timestamp('2024-08-11 00:00:00'), Timestamp('2024-08-18 00:00:00'), Timestamp('2024-08-25 00:00:00'), Timestamp('2024-09-01 00:00:00'), Timestamp('2024-09-08 00:00:00'), Timestamp('2024-09-15 00:00:00'), Timestamp('2024-09-22 00:00:00'), Timestamp('2024-09-29 00:00:00'), Timestamp('2024-10-06 00:00:00'), Timestamp('2024-10-13 00:00:00'), Timestamp('2024-10-20 00:00:00'), Timestamp('2024-10-27 00:00:00'), Timestamp('2024-11-03 00:00:00'), Tim

,work_date,department,total_overtime
317,2025-05-18,Sales,1.57
316,2025-05-18,Marketing,3.49
314,2025-05-18,Inventory,2.89
315,2025-05-18,IT,1.13
313,2025-05-18,HR,1.92
312,2025-05-18,Finance,1.60
311,2025-05-11,Sales,0.00
310,2025-05-11,Marketing,1.99
308,2025-05-11,Inventory,8.03
309,2025-05-11,IT,12.85


### 3.1. Evaluación de Estacionaridad (Dickey-Fuller Test)

In [142]:
def Prueba_Dickey_Fuller(series, department, column_name):
    """
    Realiza la prueba de Dickey-Fuller para analizar estacionaridad
    """
    print(f'\nAnálisis de Estacionalidad para {department}')
    print(f'Resultados de la prueba de Dickey-Fuller para columna: {column_name}')
    
    # Realizar la prueba de Dickey-Fuller
    dftest = adfuller(series, autolag='AIC')
    
    # Crear una serie de pandas con los resultados principales
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', 'No Lags Used', 
                                            'Número de observaciones utilizadas'])
    
    # Agregar los valores críticos al resultado
    for key, value in dftest[4].items():
        dfoutput[f'Critical Value ({key})'] = value
    
    # Mostrar los resultados
    print(dfoutput)
    
    # Interpretar los resultados
    if dftest[1] <= 0.05:
        conclusion = "Los datos son estacionarios, no requieren diferenciación"
        decision = "Rechazar la hipótesis nula"
    else:
        conclusion = "Los datos no son estacionarios, requieren diferenciación"
        decision = "No se puede rechazar la hipótesis nula"
    
    print("\nConclusión:====>")
    print(decision)
    print(conclusion)
    
    return {
        'department': department,
        'test_statistic': dftest[0],
        'p_value': dftest[1],
        'is_stationary': dftest[1] <= 0.05
    }

# Realizar análisis de estacionaridad para cada departamento
adf_results = []
for department in historical_data['department'].unique():
    dept_data = historical_data[historical_data['department'] == department]
    
    if len(dept_data) < 10:
        print(f"\nAdvertencia: Datos insuficientes para {department}")
        continue
        
    result = Prueba_Dickey_Fuller(
        dept_data["total_overtime"],
        department,
        "total_overtime"
    )
    adf_results.append(result)

# Crear DataFrame con resultados
adf_df = pd.DataFrame(adf_results)
print("\nResumen de Estacionaridad por Departamento:")
display(adf_df)

def plot_single_department(historical_data, department):
    """Visualiza tendencia para un departamento específico"""
    dept_data = historical_data[historical_data['department'] == department]
    
    # Crear figura
    fig = go.Figure()
    
    # Ajustar línea de tendencia
    z = np.polyfit(range(len(dept_data)), dept_data['total_overtime'], 1)
    trend = np.poly1d(z)
    
    # Datos históricos
    fig.add_trace(
        go.Scatter(
            x=dept_data['work_date'],
            y=dept_data['total_overtime'],
            name='Histórico',
            mode='lines+markers',
            line=dict(color='blue')
        )
    )
    
    # Línea de tendencia
    fig.add_trace(
        go.Scatter(
            x=dept_data['work_date'],
            y=trend(range(len(dept_data))),
            name='Tendencia',
            mode='lines',
            line=dict(color='red', dash='dash')
        )
    )
    
    # Actualizar diseño
    fig.update_layout(
        title=f'Tendencia de Horas Extra - {department}',
        xaxis_title="Fecha",
        yaxis_title="Horas Extra",
        template='plotly_white',
        showlegend=True,
        height=400,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.4,
            xanchor="center",
            x=0.5
        )
    )
    
    return fig

# Generar y mostrar visualizaciones individuales
print("\nGenerando visualizaciones individuales...")
for department in historical_data['department'].unique():
    fig = plot_single_department(historical_data, department)
    fig.show()
    print(f"Visualización completada para {department}")

print("\nTodas las visualizaciones han sido generadas.")



Análisis de Estacionalidad para Finance
Resultados de la prueba de Dickey-Fuller para columna: total_overtime
Test Statistic                        -5.567976
p-value                                0.000001
No Lags Used                           0.000000
Número de observaciones utilizadas    52.000000
Critical Value (1%)                   -3.562879
Critical Value (5%)                   -2.918973
Critical Value (10%)                  -2.597393
dtype: float64

Conclusión:====>
Rechazar la hipótesis nula
Los datos son estacionarios, no requieren diferenciación

Análisis de Estacionalidad para HR
Resultados de la prueba de Dickey-Fuller para columna: total_overtime
Test Statistic                        -4.898444
p-value                                0.000035
No Lags Used                           2.000000
Número de observaciones utilizadas    50.000000
Critical Value (1%)                   -3.568486
Critical Value (5%)                   -2.921360
Critical Value (10%)                  -2.5

,department,test_statistic,p_value,is_stationary
0,Finance,-5.567976,1.489681e-06,True
1,HR,-4.898444,3.511837e-05,True
2,Inventory,-2.117701,2.374569e-01,False
3,IT,-2.881014,4.760082e-02,True
4,Marketing,-6.174558,6.679154e-08,True
5,Sales,-3.986926,1.480539e-03,True



Generando visualizaciones individuales...


Visualización completada para Finance


Visualización completada para HR


Visualización completada para Inventory


Visualización completada para IT


Visualización completada para Marketing


Visualización completada para Sales

Todas las visualizaciones han sido generadas.


### 3.2. Evaluar Estacionalidad

In [143]:
def analyze_seasonality(historical_data, department=None):
    """Analiza la estacionalidad para un departamento específico usando ACF"""
    
    if department is None:
        departments = historical_data['department'].unique()
    else:
        departments = [department]
        
    # Crear figura para ACF
    fig = go.Figure()
    
    # Diccionario para almacenar resultados
    seasonality_results = {}
    
    # Analizar el departamento
    dept_data = historical_data[historical_data['department'] == department]
    
    if len(dept_data) < 10:
        print(f"\nAdvertencia: Datos insuficientes para {department}")
        return None, None
        
    # Calcular ACF
    acf_values = acf(dept_data['total_overtime'], nlags=52, fft=True)
    confidence_level = 1.96/np.sqrt(len(dept_data))
    lags = np.arange(len(acf_values))
    
    # Agregar ACF plot
    fig.add_trace(
        go.Scatter(
            x=lags,
            y=acf_values,
            mode='lines',
            name='ACF',
            line=dict(color='blue')
        )
    )
    
    # Agregar líneas de confianza
    fig.add_trace(
        go.Scatter(
            x=lags,
            y=[confidence_level]*len(lags),
            mode='lines',
            line=dict(dash='dash', color='red'),
            name='Límite de confianza superior'
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=lags,
            y=[-confidence_level]*len(lags),
            mode='lines',
            line=dict(dash='dash', color='red'),
            name='Límite de confianza inferior'
        )
    )
    
    # Detectar estacionalidad
    significant_lags = [i for i in range(1, len(acf_values)) 
                      if abs(acf_values[i]) > confidence_level]
    
    # Buscar patrones repetitivos
    if len(significant_lags) > 0:
        potential_seasons = [lag for lag in significant_lags if lag > 4]
        seasonal_strength = np.mean([abs(acf_values[lag]) for lag in potential_seasons]) if potential_seasons else 0
        
        # Determinar tipo de estacionalidad
        if seasonal_strength > 0.6:
            seasonality_type = "Fuerte"
        elif seasonal_strength > 0.3:
            seasonality_type = "Moderada"
        elif seasonal_strength > 0.1:
            seasonality_type = "Débil"
        else:
            seasonality_type = "No detectada"
            
        # Identificar posible período estacional
        if potential_seasons:
            possible_period = min(potential_seasons)
        else:
            possible_period = None
    else:
        seasonality_type = "No detectada"
        possible_period = None
        seasonal_strength = 0
    
    # Almacenar resultados
    seasonality_results = {
        'seasonality_type': seasonality_type,
        'seasonal_strength': seasonal_strength,
        'possible_period': possible_period,
        'significant_lags': significant_lags
    }
    
    # Actualizar diseño con leyenda abajo al centro
    fig.update_layout(
        height=400,
        title_text=f"Análisis de Autocorrelación - {department}",
        showlegend=True,
        template='plotly_white',
        xaxis_title="Lags",
        yaxis_title="Autocorrelación",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.4,
            xanchor="center",
            x=0.5
        )
    )
    
    # Mostrar resultados
    print(f"\nResultados del análisis de estacionalidad para {department}:")
    print("-" * 70)
    print(f"Tipo de estacionalidad: {seasonality_type}")
    print(f"Fuerza de la estacionalidad: {seasonal_strength:.2f}")
    if possible_period:
        print(f"Posible período estacional: {possible_period} semanas")
    print(f"Lags significativos: {significant_lags}")
    
    return fig, seasonality_results

# Para usar la función para un departamento específico:
departments = historical_data['department'].unique()
for dept in departments:
    print(f"\nAnalizando {dept}...")
    fig_acf, results = analyze_seasonality(historical_data, dept)
    if isinstance(fig_acf, go.Figure):
        fig_acf.show()



Analizando Finance...



Resultados del análisis de estacionalidad para Finance:
----------------------------------------------------------------------
Tipo de estacionalidad: No detectada
Fuerza de la estacionalidad: 0.00
Lags significativos: []



Analizando HR...

Resultados del análisis de estacionalidad para HR:
----------------------------------------------------------------------
Tipo de estacionalidad: No detectada
Fuerza de la estacionalidad: 0.00
Lags significativos: []



Analizando Inventory...

Resultados del análisis de estacionalidad para Inventory:
----------------------------------------------------------------------
Tipo de estacionalidad: No detectada
Fuerza de la estacionalidad: 0.00
Lags significativos: []



Analizando IT...

Resultados del análisis de estacionalidad para IT:
----------------------------------------------------------------------
Tipo de estacionalidad: Moderada
Fuerza de la estacionalidad: 0.35
Posible período estacional: 7 semanas
Lags significativos: [2, 7]



Analizando Marketing...

Resultados del análisis de estacionalidad para Marketing:
----------------------------------------------------------------------
Tipo de estacionalidad: Débil
Fuerza de la estacionalidad: 0.27
Posible período estacional: 12 semanas
Lags significativos: [12]



Analizando Sales...

Resultados del análisis de estacionalidad para Sales:
----------------------------------------------------------------------
Tipo de estacionalidad: Débil
Fuerza de la estacionalidad: 0.27
Posible período estacional: 5 semanas
Lags significativos: [1, 5]


## 4. Entrenar Modelo SARIMA

Crear y entrenar modelo SARIMA para cada departamento.

In [144]:


def calculate_smape(y_true, y_pred):
    """Calcula SMAPE para manejar valores cero."""
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    non_zero_mask = denominator != 0
    if np.any(non_zero_mask):
        smape = np.mean(np.abs(y_true[non_zero_mask] - y_pred[non_zero_mask]) / 
                        denominator[non_zero_mask]) * 100
        return min(smape, 100)
    return np.mean(np.abs(y_true - y_pred))

def calculate_mase(y_true, y_pred, train_data, period=1):
    """Calcula MASE comparando con un modelo ingenuo (shifted)."""
    naive_forecast = train_data.shift(period).dropna()
    naive_error = np.abs(naive_forecast - train_data[period:])
    mean_naive_error = np.mean(naive_error) if len(naive_error) > 0 else np.inf
    mae = np.mean(np.abs(y_true - y_pred))
    return mae / mean_naive_error if mean_naive_error != 0 else np.inf

def evaluate_model_quality(metrics, residuals, test_size):
    """
    Evalúa la calidad del modelo según métricas y residuos.
    Retorna una clasificación: 'Bueno', 'Aceptable' o 'Pobre'.
    """
    rmse = metrics['rmse']
    mae = metrics['mae']
    smape = metrics['smape']
    mase = metrics['mase']
    
    lb_test = acorr_ljungbox(residuals, lags=[min(4, len(residuals)-1)], return_df=True)
    lb_pvalue = lb_test['lb_pvalue'].iloc[0] if len(lb_test) > 0 else 0
    residual_mean = np.mean(residuals)
    
    is_good = (
        rmse < 5.0 and
        mae < 4.0 and
        smape < 60 and
        mase < 1.0 and
        lb_pvalue > 0.05 and
        abs(residual_mean) < 0.5
    )
    
    is_acceptable = (
        rmse < 7.0 and
        mae < 6.0 and
        smape < 85 and
        mase < 1.5 and
        lb_pvalue > 0.01 and
        abs(residual_mean) < 1.0
    )
    
    if is_good:
        return "Bueno"
    elif is_acceptable:
        return "Aceptable"
    else:
        return "Pobre"

def train_sarimax_model(dept_data, exog_cols=None):
    """
    Entrena modelo SARIMAX con Auto-ARIMA ajustando estacionalidad según análisis previo.
    """
    # Preparar datos 
    df = dept_data[['work_date', 'total_overtime'] + (exog_cols if exog_cols else [])].copy()
    df.set_index('work_date', inplace=True)
    
    # Dividir datos en entrenamiento (80%) y prueba (20%)
    train_size = int(len(df) * 0.8)
    train = df[:train_size]
    test = df[train_size:]
    
    # Preparar variables exógenas si existen
    exog_train = train[exog_cols] if exog_cols else None
    exog_test = test[exog_cols] if exog_cols else None
    
    # Obtener análisis de estacionalidad previo
    _, seasonality_results = analyze_seasonality(historical_data, dept_data['department'].iloc[0])
    
    # Determinar parámetros de estacionalidad
    has_seasonality = seasonality_results['seasonality_type'] in ["Fuerte", "Moderada"]
    seasonal_period = seasonality_results['possible_period'] if has_seasonality else 1
    
    # Verificar datos faltantes y otras validaciones
    # ... resto del código de validación ...
    
    # Log de configuración del modelo
    print(f"\nConfigurando modelo SARIMA para {dept_data['department'].iloc[0]}:")
    print(f"Estacionalidad detectada: {seasonality_results['seasonality_type']}")
    print(f"Fuerza estacional: {seasonality_results['seasonal_strength']:.2f}")
    if has_seasonality:
        print(f"Período estacional: {seasonal_period} semanas")
    
    # Encontrar mejores parámetros con Auto-ARIMA
    try:
        # Configuración base para no estacional
        arima_params = {
            'start_p': 0, 
            'd': None,  # Permitir que auto_arima determine d
            'start_q': 0,
            'max_p': 4, 
            'max_d': 2, 
            'max_q': 4,
            'seasonal': False,
            'm': 1
        }
        
        # Ajustar configuración si hay estacionalidad
        if has_seasonality:
            arima_params.update({
                'seasonal': True,
                'm': seasonal_period,
                'start_P': 0,
                'D': None,  # Permitir que auto_arima determine D
                'start_Q': 0,
                'max_P': 2,
                'max_D': 1,
                'max_Q': 2
            })
        
        # Entrenar modelo Auto-ARIMA
        modelo_auto = auto_arima(
            train['total_overtime'],
            exogenous=exog_train,
            error_action='warn',
            trace=False,
            suppress_warnings=True,
            stepwise=True,
            random_state=20,
            n_fits=50,
            **arima_params
        )
        
        # Obtener orden del modelo
        order = modelo_auto.order
        seasonal_order = modelo_auto.seasonal_order if has_seasonality else (0, 0, 0, 0)
        
        # Log de órdenes seleccionados
        print(f"Orden ARIMA seleccionado: {order}")
        if has_seasonality:
            print(f"Orden estacional seleccionado: {seasonal_order}")
        
        # Entrenar modelo final SARIMAX
        final_model = SARIMAX(
            df['total_overtime'],
            exog=df[exog_cols] if exog_cols else None,
            order=order,
            seasonal_order=seasonal_order
        )
        model_fit = final_model.fit(disp=False)
        
        # Calcular predicciones y residuos
        predictions = model_fit.get_prediction(start=train_size)
        residuals = df['total_overtime'][train_size:] - predictions.predicted_mean
        
        # Calcular métricas
        mae = np.mean(np.abs(residuals))
        rmse = np.sqrt(np.mean(residuals**2))
        smape = calculate_smape(df['total_overtime'][train_size:], predictions.predicted_mean)
        mase = calculate_mase(df['total_overtime'][train_size:], predictions.predicted_mean, 
                            df['total_overtime'][:train_size])
        
        # Crear diccionario de métricas
        metrics = {
            'order': order,
            'seasonal_order': seasonal_order,
            'mae': mae,
            'rmse': rmse,
            'smape': smape,
            'mase': mase,
            'quality': evaluate_model_quality({
                'mae': mae,
                'rmse': rmse,
                'smape': smape,
                'mase': mase
            }, residuals, len(df) - train_size)
        }
        
        return model_fit, metrics
        
    except Exception as e:
        logging.error(f"Error entrenando modelo SARIMAX para {dept_data['department'].iloc[0]}: {str(e)}")
        return None, {'error': f"Excepción: {str(e)}"}

def add_exogenous_variables(df):
    """
    Agrega variables exógenas de ejemplo (is_weekend, month).
    Modificar según variables reales disponibles.
    """
    df = df.copy()
    df['is_weekend'] = df['work_date'].dt.dayofweek.isin([5, 6]).astype(int)
    df['month'] = df['work_date'].dt.month
    return df

# Reiniciar variables de almacenamiento
department_models = {}
metrics = {}
department_forecasts = {}

# Agregar variables exógenas al DataFrame
historical_data = add_exogenous_variables(historical_data)

# Definir columnas exógenas
exog_cols = ['is_weekend', 'month']  # Ajustar según variables reales

print("Iniciando entrenamiento de modelos SARIMAX...")

# Iterar sobre cada departamento
for department in historical_data['department'].unique():
    print(f"\nProcesando departamento: {department}")
    dept_data = historical_data[historical_data['department'] == department]
    
    if len(dept_data) < 10:
        print(f"Advertencia: Datos insuficientes para {department}")
        continue
    
    # Entrenar modelo SARIMAX
    model, model_metrics = train_sarimax_model(dept_data, exog_cols=exog_cols)
    
    if model is None:
        print(f"Error: No se pudo entrenar el modelo para {department}: {model_metrics.get('error', 'Error desconocido')}")
        continue
        
    # Almacenar resultados
    department_models[department] = model
    metrics[department] = model_metrics
    
    # Mostrar resultados
    print(f"\nResultados para {department}:")
    print(f"Orden SARIMA: {model_metrics['order']}")
    print(f"Orden Seasonal: {model_metrics['seasonal_order']}")
    print(f"RMSE: {model_metrics['rmse']:.2f}")
    print(f"MAE: {model_metrics['mae']:.2f}")
    print(f"SMAPE: {model_metrics['smape']:.2f}%")
    print(f"MASE: {model_metrics['mase']:.2f}")
    print(f"Calidad del modelo: {model_metrics['quality']}")

Iniciando entrenamiento de modelos SARIMAX...

Procesando departamento: Finance

Resultados del análisis de estacionalidad para Finance:
----------------------------------------------------------------------
Tipo de estacionalidad: No detectada
Fuerza de la estacionalidad: 0.00
Lags significativos: []

Configurando modelo SARIMA para Finance:
Estacionalidad detectada: No detectada
Fuerza estacional: 0.00
Orden ARIMA seleccionado: (1, 0, 0)

Resultados para Finance:
Orden SARIMA: (1, 0, 0)
Orden Seasonal: (0, 0, 0, 0)
RMSE: 3.36
MAE: 2.80
SMAPE: 51.90%
MASE: 1.09
Calidad del modelo: Aceptable

Procesando departamento: HR

Resultados del análisis de estacionalidad para HR:
----------------------------------------------------------------------
Tipo de estacionalidad: No detectada
Fuerza de la estacionalidad: 0.00
Lags significativos: []

Configurando modelo SARIMA para HR:
Estacionalidad detectada: No detectada
Fuerza estacional: 0.00
Orden ARIMA seleccionado: (3, 1, 1)

Resultados para H

## 5. Generar Predicciones

Predicción para las próximas 4 semanas de Overtime

In [145]:


def generate_predictions_sarimax(model, last_date, dept_data, exog_cols=None, periods=4, freq='W'):
  
    # Validar entradas
    if model is None:
        return pd.DataFrame(), f"Modelo no válido para el departamento"
    
    if not isinstance(last_date, pd.Timestamp):
        try:
            last_date = pd.to_datetime(last_date)
        except:
            return pd.DataFrame(), f"Fecha inválida: {last_date}"
    
    if periods < 1:
        return pd.DataFrame(), f"Número de períodos inválido: {periods}"
    
    # Generar fechas futuras
    future_dates = pd.date_range(start=last_date, periods=periods+1, freq=freq)[1:]
    
    # Preparar variables exógenas futuras
    if exog_cols:
        # Crear DataFrame para variables exógenas futuras
        future_exog = pd.DataFrame(index=future_dates)
        future_exog['is_weekend'] = future_exog.index.dayofweek.isin([5, 6]).astype(int)
        future_exog['month'] = future_exog.index.month
        
        # Verificar que todas las columnas exógenas estén presentes
        missing_cols = [col for col in exog_cols if col not in future_exog.columns]
        if missing_cols:
            return pd.DataFrame(), f"Columnas exógenas faltantes: {missing_cols}"
    else:
        future_exog = None
    
    # Obtener predicciones y sus intervalos de confianza
    try:
        forecast = model.get_forecast(steps=periods, exog=future_exog)
        mean_forecast = forecast.predicted_mean
        confidence_int = forecast.conf_int()
        
        # Truncar predicciones negativas a cero
        mean_forecast = np.maximum(mean_forecast, 0)
        confidence_int.iloc[:, 0] = np.maximum(confidence_int.iloc[:, 0], 0)  # yhat_lower
        confidence_int.iloc[:, 1] = np.maximum(confidence_int.iloc[:, 1], 0)  # yhat_upper
        
        # Crear DataFrame con las predicciones
        predictions_df = pd.DataFrame({
            'ds': future_dates,
            'yhat': mean_forecast,
            'yhat_lower': confidence_int.iloc[:, 0],
            'yhat_upper': confidence_int.iloc[:, 1]
        })
        
        return predictions_df, None
    
    except Exception as e:
        return pd.DataFrame(), f"Error generando predicciones: {str(e)}"

# Generar predicciones para cada departamento
department_forecasts = {}
print("\nGenerando predicciones para las próximas 4 semanas...")

for department, model in department_models.items():
    print(f"\nProcesando departamento: {department}")
    
    # Obtener datos históricos del departamento
    dept_data = historical_data[historical_data['department'] == department]
    if dept_data.empty:
        print(f"Error: No hay datos históricos para {department}")
        continue
    
    # Obtener última fecha de datos históricos
    last_date = dept_data['work_date'].max()
    
    # Generar predicciones
    forecast, error = generate_predictions_sarimax(
        model, 
        last_date, 
        dept_data, 
        exog_cols=['is_weekend', 'month'],  # Ajustar según variables exógenas
        periods=4,
        freq='W'
    )
    
    if error:
        print(f"Error generando predicciones para {department}: {error}")
        continue
    
    # Almacenar predicciones
    department_forecasts[department] = forecast
    
    # Mostrar predicciones
    print(f"\nPredicciones para {department}:")
    print("\nFecha\t\tPredicción\tIntervalo de Confianza")
    print("-" * 60)
    for _, row in forecast.iterrows():
        print(f"{row['ds'].strftime('%Y-%m-%d')}\t{row['yhat']:.2f}\t\t({row['yhat_lower']:.2f}, {row['yhat_upper']:.2f})")

print("\nPredicciones generadas para departamentos:", list(department_forecasts.keys()))


Generando predicciones para las próximas 4 semanas...

Procesando departamento: Finance

Predicciones para Finance:

Fecha		Predicción	Intervalo de Confianza
------------------------------------------------------------
2025-05-25	4.79		(0.00, 10.70)
2025-06-01	5.29		(0.00, 11.32)
2025-06-08	5.42		(0.00, 11.46)
2025-06-15	5.45		(0.00, 11.49)

Procesando departamento: HR

Predicciones para HR:

Fecha		Predicción	Intervalo de Confianza
------------------------------------------------------------
2025-05-25	10.87		(2.82, 18.91)
2025-06-01	6.56		(0.00, 14.67)
2025-06-08	8.52		(0.41, 16.64)
2025-06-15	5.20		(0.00, 13.36)

Procesando departamento: Inventory

Predicciones para Inventory:

Fecha		Predicción	Intervalo de Confianza
------------------------------------------------------------
2025-05-25	6.26		(0.00, 13.10)
2025-06-01	5.96		(0.00, 12.80)
2025-06-08	5.96		(0.00, 12.80)
2025-06-15	5.96		(0.00, 12.80)

Procesando departamento: IT

Predicciones para IT:

Fecha		Predicción	Intervalo de

## 6. Visualización de Resultados

Datos históricos de 24 semanas más pronósticos e intervalos de confianza para las próximas 4 semanas.

In [146]:
def generate_predictions_sarima(model, last_date, periods=4):
    """Generate predictions using a SARIMA model
    
    Args:
        model: Fitted SARIMA model
        last_date: Last date in the historical data
        periods: Number of periods to forecast (default=4)
    
    Returns:
        DataFrame with predictions and confidence intervals
    """
    # Generate forecast for next 4 weeks
    forecast = model.get_forecast(steps=periods)
    
    # Get prediction dates
    prediction_dates = pd.date_range(
        start=last_date + pd.Timedelta(days=7),
        periods=periods,
        freq='W-SUN'
    )
    
    # Create DataFrame with predictions
    forecast_df = pd.DataFrame({
        'ds': prediction_dates,
        'yhat': forecast.predicted_mean.clip(lower=0),
        'yhat_lower': forecast.conf_int().iloc[:, 0].clip(lower=0),
        'yhat_upper': forecast.conf_int().iloc[:, 1].clip(lower=0)
    })
    
    forecast_df.set_index('ds', inplace=True)
    
    return forecast_df

def plot_forecast(department, historical_data, forecast):
    """Create interactive plot for a department's forecast"""
    
    # Filter historical data for department
    hist_dept = historical_data[historical_data['department'] == department]
    last_24_weeks = hist_dept.tail(24).copy()
    
    # Create figure
    fig = go.Figure()
    
    # Add historical data points
    fig.add_trace(
        go.Scatter(
            x=last_24_weeks['work_date'],
            y=last_24_weeks['total_overtime'],
            name='Datos Históricos',
            mode='markers+lines',
            line=dict(color='blue'),
            hovertemplate="<b>Fecha:</b> %{x}<br>" +
                         "<b>Valor Real:</b> %{y:.1f}<extra></extra>"
        )
    )
    
    # Get predictions for historical data
    model = department_models[department]
    historical_predictions = model.get_prediction(start=last_24_weeks['work_date'].min())
    historical_mean = historical_predictions.predicted_mean.clip(lower=0)
    
    # Add historical predictions as dashed red line
    fig.add_trace(
        go.Scatter(
            x=last_24_weeks['work_date'],
            y=historical_mean,
            name='Predicción Histórica',
            mode='lines',
            line=dict(color='red', dash='dash'),
            hovertemplate="<b>Fecha:</b> %{x}<br>" +
                         "<b>Predicción:</b> %{y:.1f}<extra></extra>"
        )
    )
    
    # Add forecast line continuous from last historical point
    fig.add_trace(
        go.Scatter(
            x=[last_24_weeks['work_date'].iloc[-1]] + forecast.index.tolist(),
            y=[historical_mean.iloc[-1]] + forecast['yhat'].tolist(),
            name='Pronóstico',
            mode='lines',
            line=dict(color='red'),
            hovertemplate="<b>Fecha:</b> %{x}<br>" +
                         "<b>Predicción:</b> %{y:.1f}<br>" +
                         "<b>IC Inferior:</b> %{customdata[0]:.1f}<br>" +
                         "<b>IC Superior:</b> %{customdata[1]:.1f}<extra></extra>",
            customdata=np.column_stack((
                [0] + forecast['yhat_lower'].tolist(),
                [0] + forecast['yhat_upper'].tolist()
            ))
        )
    )
    
    # Add confidence interval for forecast period
    fig.add_trace(
        go.Scatter(
            x=forecast.index.tolist() + forecast.index.tolist()[::-1],
            y=forecast['yhat_upper'].tolist() + forecast['yhat_lower'].tolist()[::-1],
            fill='toself',
            fillcolor='rgba(211,211,211,0.3)',
            line=dict(color='rgba(255,255,255,0)'),
            name='Intervalo de Confianza 95%',
            showlegend=True,
            hoverinfo='skip'
        )
    )
    
    # Update layout
    fig.update_layout(
        title=f'Pronóstico de Horas Extra - {department}',
        xaxis_title='Fecha',
        yaxis_title='Horas Extra',
        hovermode='x unified',
        showlegend=True,
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.4,
            xanchor="center",
            x=0.5
        )
    )
    
    return fig

# Generar predicciones para cada departamento
print("\nGenerando visualizaciones...")

for department in department_models.keys():
    try:
        # Get last date and generate predictions if needed
        dept_data = historical_data[historical_data['department'] == department]
        last_date = dept_data['work_date'].max()
        
        if department not in department_forecasts:
            forecast = generate_predictions_sarima(department_models[department], last_date)
            department_forecasts[department] = forecast
        
        # Create and display plot
        fig = plot_forecast(
            department,
            historical_data,
            department_forecasts[department]
        )
        fig.show()
        print(f"Visualización generada para {department}")
        
    except Exception as e:
        print(f"Error generando visualización para {department}: {e}")
        continue

print("\nProceso de visualización completado.")


Generando visualizaciones...


Visualización generada para Finance


Visualización generada para HR


Visualización generada para Inventory


Visualización generada para IT


Visualización generada para Marketing


Visualización generada para Sales

Proceso de visualización completado.


## 7. Guardar Modelo y Predicciones

Almacenamiento del Modelo y Resultados en MS SQL SERVER

In [147]:
def save_predictions():

    # Crear directorio si no existe
    os.makedirs('Modelos Entrenados', exist_ok=True)

    # Guardar modelo
    model_path = 'Modelos Entrenados/overtime_forecast_model.pkl'
    joblib.dump(model, model_path)
    print(f"Modelo guardado en: {model_path}")

    # Guardar predicciones y métricas en la base de datos
    conn = get_db_connection()
    cursor = conn.cursor()
    timestamp = datetime.datetime.now()

    try:
        # 1. Crear tabla Overtime_Predictions si no existe
        cursor.execute("""
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'Overtime_Predictions')
            CREATE TABLE Overtime_Predictions (
                id INT IDENTITY(1,1) PRIMARY KEY,
                timestamp DATETIME,
                department VARCHAR(100),
                prediction_date DATE,
                predicted_value FLOAT,
                confidence_lower FLOAT,
                confidence_upper FLOAT
            )
        """)
        
        # 2. Crear tabla ML_Model_Metrics_Overtime_Predictions si no existe 
        cursor.execute("""
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'ML_Model_Metrics_Overtime_Predictions')
            CREATE TABLE ML_Model_Metrics_Overtime_Predictions (
                id INT IDENTITY(1,1) PRIMARY KEY,
                timestamp DATETIME,
                department VARCHAR(100),
                rmse FLOAT,
                mae FLOAT,
                smape FLOAT,
                mase FLOAT,
                model_quality VARCHAR(50)
            )
        """)
        
        # Guardar predicciones para cada departamento
        for department, forecast in department_forecasts.items():
            # Insertar predicciones
            for _, row in forecast.iterrows():
                cursor.execute("""
                    INSERT INTO Overtime_Predictions 
                    (timestamp, department, prediction_date, predicted_value, 
                     confidence_lower, confidence_upper)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (
                    timestamp,
                    department, 
                    row['ds'],
                    row['yhat'],
                    row['yhat_lower'],
                    row['yhat_upper']
                ))
            
            # Insertar métricas del modelo
            metric = metrics[department]
            cursor.execute("""
                INSERT INTO ML_Model_Metrics_Overtime_Predictions 
                (timestamp, department, rmse, mae, smape, mase, model_quality)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                timestamp,
                department,
                metric['rmse'],
                metric['mae'],
                metric['smape'],
                metric['mase'],
                metric['quality']
            ))
        
        conn.commit()
        logging.info("Predicciones y métricas guardadas exitosamente")
        
        # Mostrar resumen de lo guardado
        print("\nResumen de predicciones guardadas:")
        print("-" * 70)
        for department in department_forecasts.keys():
            print(f"\nDepartamento: {department}")
            print("Fecha Predicha\t\tPredicción\tIntervalo de Confianza")
            print("-" * 60)
            
            forecast = department_forecasts[department]
            for _, row in forecast.iterrows():
                print(f"{row['ds'].strftime('%Y-%m-%d')}\t{row['yhat']:.2f}\t\t({row['yhat_lower']:.2f}, {row['yhat_upper']:.2f})")
            
            # Mostrar métricas
            metric = metrics[department]
            print(f"\nMétricas del modelo:")
            print(f"RMSE: {metric['rmse']:.2f}")
            print(f"MAE: {metric['mae']:.2f}")
            print(f"SMAPE: {metric['smape']:.2f}%")
            print(f"MASE: {metric['mase']:.2f}")
            print(f"Calidad del modelo: {metric['quality']}")
            print("-" * 70)
        
    except Exception as e:
        conn.rollback()
        logging.error(f"Error guardando datos: {e}")
        raise
    finally:
        conn.close()

# Ejecutar el guardado de predicciones
try:
    save_predictions()
    print("\nDatos guardados exitosamente en la base de datos.")
except Exception as e:
    print(f"\nError al guardar los datos: {str(e)}")


def verify_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        # Execute queries separately to get correct results
        cursor.execute("SELECT COUNT(*) FROM Overtime_Predictions")
        pred_count = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) FROM ML_Model_Metrics_Overtime_Predictions")
        metrics_count = cursor.fetchone()[0]
        
        print(f"\nRegistros en Overtime_Predictions: {pred_count}")
        print(f"Registros en ML_Model_Metrics: {metrics_count}")
    except Exception as e:
        print(f"Error verificando tablas: {e}")
    finally:
        conn.close()

# Verificar tablas
verify_tables()    

Modelo guardado en: Modelos Entrenados/overtime_forecast_model.pkl


INFO:root:Predicciones y métricas guardadas exitosamente



Resumen de predicciones guardadas:
----------------------------------------------------------------------

Departamento: Finance
Fecha Predicha		Predicción	Intervalo de Confianza
------------------------------------------------------------
2025-05-25	4.79		(0.00, 10.70)
2025-06-01	5.29		(0.00, 11.32)
2025-06-08	5.42		(0.00, 11.46)
2025-06-15	5.45		(0.00, 11.49)

Métricas del modelo:
RMSE: 3.36
MAE: 2.80
SMAPE: 51.90%
MASE: 1.09
Calidad del modelo: Aceptable
----------------------------------------------------------------------

Departamento: HR
Fecha Predicha		Predicción	Intervalo de Confianza
------------------------------------------------------------
2025-05-25	10.87		(2.82, 18.91)
2025-06-01	6.56		(0.00, 14.67)
2025-06-08	8.52		(0.41, 16.64)
2025-06-15	5.20		(0.00, 13.36)

Métricas del modelo:
RMSE: 3.20
MAE: 2.63
SMAPE: 41.08%
MASE: 0.46
Calidad del modelo: Bueno
----------------------------------------------------------------------

Departamento: Inventory
Fecha Predicha		Predic